In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# Ensure no W&B logging will be performed
sys.argv = "main.py -log tb -name tst -reset 1 -lm.eval.enable 0 -log tb -batch_size 20 -restore /cm/shared/anonymous/moeut_training_code/save/slimpajama_moe_no_attmoe_154M_standard_lb/checkpoint/model-100000.pth".split(" ")

# Pretend we are in the main directory
os.chdir("../../")
import os
os.environ["MOE_TYPE"] = "moe_layer"
os.environ.get("MOE_TYPE", "moe_layer")

'moe_layer'

In [2]:
from main import initialize
import torch
import torch.nn.functional as F
from layers import MoE
import matplotlib.pyplot as plt
import pickle
import numpy as np

%matplotlib inline

plt.rcParams['text.usetex'] = False #Let TeX do the typsetting
plt.rcParams['text.latex.preamble'] = '\\usepackage{sansmath}\n\\sansmath' #Force sans-serif math mode (for axes labels)
plt.rcParams['font.family'] = 'DejaVu Sans' # Change from 'Arial' to 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial' # Choose a nice font here

plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

print(MoE)

/cm/archive/anonymous/miniconda3/envs/moeut/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-04 10:29:47,360] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/cm/archive/anonymous/miniconda3/envs/moeut/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/cm/archive/anonymous/miniconda3/envs/moeut/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


<class 'layers.moe_layer.MoE'>


In [3]:
helper, task = initialize()
task.create_data_fetcher()

orig_run_model_valid = task.run_model_validation

Restoring: /cm/shared/anonymous/moeut_training_code/save/slimpajama_moe_no_attmoe_154M_standard_lb/checkpoint/model-30000.pth...
No distributed environment detected
Resetting training state...
Port already used: 7000
Port already used: 7001
Port already used: 7002
Found tensorboard in /cm/archive/anonymous/miniconda3/envs/moeut/bin/tensorboard
Starting Tensorboard server on 7003
ERROR: failed to start Tensorboard server. Server not responding.
Available GPUs: 1
SlimPajama: Generating map...
Map done.
SlimPajama: Loaded tokenizer.
SlimPajama: 58345 chunks missing
Token limit reached. No need to tokenize more.
SlimPajama: Limiting to first 164 chunks because limited to 2050048000 tokens
SlimPajama: Loaded tokenizer.
SlimPajama: 31345 chunks missing
Token limit reached. No need to tokenize more.
SlimPajama: Limiting to first 59 chunks because limited to 1032192 tokens
Weight info:
  n_model_weights: 149733376
  n_attention_weights: 10747904
  n_non_attnetion_weights: 138985472
  attention

/cm/shared/anonymous/moeut_training_code/tasks/lm_base.py:115: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  data = torch.tensor(res, dtype=torch.long, device=self.helper.device).T


In [4]:
nexp = task.helper.args.moe.n_experts
ntok = task.helper.args.sentencepiece.n_pieces
nlayers = task.helper.args.transformer.encoder_n_layers
ngrp = 16

token_counts = 0

counts = torch.zeros(ngrp, nlayers // ngrp, nexp, ntok)

In [5]:
counts.size()

torch.Size([16, 1, 66, 8000])

In [6]:
global this_data

def run_model_validation(self, data):
    global token_counts
    global this_data

    token_counts = token_counts + F.one_hot(data["data"].flatten().long(), ntok).sum(0)

    this_data = data
    return orig_run_model_valid(data)

task.run_model_validation = run_model_validation.__get__(task)
task

In [7]:
id_map = {}

def patch_module(module):

    myid = id(module)
    if myid in id_map:
        return

    gid = len(id_map)
    id_map[myid] = gid

    # sel_val, sel_index = self.topk(

    def new_topk(self, *args, **kwargs):
        nonlocal gid
        global this_data
        data = this_data["data"][:-1].T

        sel_val, sel_index = MoE.topk(self, *args, **kwargs)

        assert data.shape == sel_index.shape[:-1]

        data = data.reshape(-1)

        # Shape of counts[gid]: nexp, ntok
        # Linear index: expert * ntok + tok

        seli = sel_index.flatten(end_dim=-2) * ntok
        addi = seli + data[..., None]
        addi = addi.flatten().cpu()

        counts[gid][self.layer // ngrp].flatten().index_add_(0, addi, torch.ones_like(addi, dtype=torch.float32))

        return sel_val, sel_index


    module.topk = new_topk.__get__(module)

In [8]:
for m in task.model.modules():
    if isinstance(m, MoE):
        patch_module(m)

In [9]:
task.validate()

Starting validation on val...


  0%|          | 0/100 [00:00<?, ?it/s]

New shape: (torch.Size([10, 1024, 512]), torch.Size([66, 512, 128]))
New shape: (torch.Size([10, 1024, 8, 128]), torch.Size([66, 128, 512]))


100%|██████████| 100/100 [03:30<00:00,  2.10s/it]

Validation done on worker 0.
Validation accuracy on val: 0.4498486328125


{'val/loss': 2.812581647872925,
 'val/accuracy': 0.4498486328125,
 'val/perplexity': 16.652854578259007,
 'val/time_since_best_loss': 0,
 'val/time_since_best_accuracy': 0,
 'mean_accuracy': 0.4498486328125,
 'mean_loss': 2.8125816559791565}

In [10]:
order = torch.argsort(token_counts, descending=True).cpu()
token_counts_o = token_counts.cpu()[order]
counts_o = counts[:, :, :, order]

In [11]:
counts_o[0][:,:,0].squeeze().size()

torch.Size([66])

In [12]:
# Add this code to decode tokens
def decode_tokens(token_ids):
    """Decode token IDs to their string representations"""
    # Use the vocabulary from the task's train set
    decoded_tokens = []
    for token_id in token_ids:
        token = task.train_set.vocabulary.to_string([token_id])
        decoded_tokens.append(token)
    return decoded_tokens

ostart = 0
count = 20
gid = 15
layer = 0

# Get the token IDs
token_ids = order[ostart:ostart+count].tolist()

# Decode the tokens
decoded_tokens = decode_tokens(token_ids)

# Print token IDs and their decoded values
for i, (token_id, token_text) in enumerate(zip(token_ids, decoded_tokens)):
    print(f"Token {i}: ID={token_id}, Text='{token_text}'")

Token 0: ID=7857, Text='.'
Token 1: ID=7859, Text=','
Token 2: ID=10, Text='the'
Token 3: ID=35, Text='of'
Token 4: ID=33, Text='and'
Token 5: ID=31, Text='to'
Token 6: ID=7836, Text=''
Token 7: ID=4, Text='a'
Token 8: ID=32, Text='in'
Token 9: ID=7844, Text='s'
Token 10: ID=7869, Text='1'
Token 11: ID=7868, Text='''
Token 12: ID=7866, Text='0'
Token 13: ID=7870, Text='-'
Token 14: ID=7874, Text='2'
Token 15: ID=64, Text='is'
Token 16: ID=96, Text='that'
Token 17: ID=66, Text='for'
Token 18: ID=29, Text='ed'
Token 19: ID=70, Text='on'


In [13]:
counts_o.size()

torch.Size([16, 1, 66, 8000])

In [14]:
labels = task.train_set.vocabulary(order[ostart:ostart+count].tolist())
save_experts_assign = np.zeros((counts_o.size(0), counts_o.size(2), counts_o.size(3)))

for layer in range(counts_o.size(0)):
    counts_o_layer = counts_o[layer, 0, :, :]

    # get top_k
    score, index = counts_o_layer.topk(6, dim=-1, sorted=False)
    score = score / (score.sum(dim=-1, keepdim=True) + 1e-20)
    counts_o_layer_topk = torch.zeros_like(counts_o_layer)
    counts_o_layer_topk = counts_o_layer_topk.scatter(-1, index, score)

    save_experts_assign[layer] = counts_o_layer_topk.numpy()

In [15]:
np.save(f"/cm/shared/anonymous/moeut_training_code/paper/deepseek/router_saturation/smoe/{sys.argv[-1].split('/')[-1].split('.')[0]}.npy", save_experts_assign)